# <center>Daily Bucket Report </center>
- *by 홍석진, 이준혁*
---
> <span style = "color:gray"> 2022.04.25　:　초안 작성 </sapn> <br>
> <span style = "color:gray"> 2022.05.02　:　자동화 및 slack 전송 코드 작성 </sapn> <br>
> <span style = "color:red"> 2022.05.06　:　실험플랫폼 모듈화  </sapn> <br>
> <span style = "color:red">　　　　　　　ㄴ 준혁 : data input / MLS update </sapn>

---

### 000_Import Library

In [ ]:
#hsj

In [ ]:
from sktmls.experiments.experiment import ExperimentClient

import pandas as pd

import numpy as np

import plotly.graph_objs as go

from scipy.stats import beta, bernoulli

In [ ]:
pip install -U kaleido

In [ ]:
config = get_mls_config(env='prd', user='scm')
experiment_client = ExperimentClient(**config)
experiment_client

In [ ]:
experiment_info = experiment_client.get_experiment(name = 'scm_reco_subsc_prod')
experiment_info

In [ ]:
experiment_info.channels

In [ ]:
channels_info = experiment_info.get()['channels']
channels_list = list(map(lambda x: x.get()['screen_id'], channels_info))
channels_list

In [ ]:
key = ['channel_id', 'bucket_name', 'dt']
key_ = ['channel_id', 'bucket_name']

In [ ]:
stat = df.reset_index().sort_values(key).reset_index(drop=True).copy(deep=True)

In [ ]:
stat['denominator_w7'] = stat.groupby(key_)['denominator'].transform(lambda x: x.rolling(window=7, min_periods=1, center=False).sum())
stat['numerator_w7'] = stat.groupby(key_)['numerator'].transform(lambda x: x.rolling(window=7, min_periods=1, center=False).sum())

stat['denominator_w14'] = stat.groupby(key_)['denominator'].transform(lambda x: x.rolling(window=14, min_periods=1, center=False).sum())
stat['numerator_w14'] = stat.groupby(key_)['numerator'].transform(lambda x: x.rolling(window=14, min_periods=1, center=False).sum())
#
# stat['click_cnt_w7'] = stat.groupby(key_)['click_cnt'].transform(lambda x: x.rolling(window=7, min_periods=1, center=False).sum())
# stat['scrb_cnt_w7'] = stat.groupby(key_)['scrb_cnt'].transform(lambda x: x.rolling(window=7, min_periods=1, center=False).sum())

In [ ]:
stat['CTR_w7'] = stat['numerator_w7'] / stat['denominator_w7']
stat['CTR_w14'] = stat['numerator_w14'] / stat['denominator_w14']

In [ ]:

class mls_thompson(ExperimentClient) :
    def __init__(self, env, runtime_env, username, password, experiment_name, base_ratio = 10) :
        self.env = env
        self.runtime_env = runtime_env
        self.username = username
        self.password = password
        self.experiment_name = experiment_name
        self.base_ratio = base_ratio
        
        print(self.experiment_name)
        super(mls_thompson, self).__init__(self.env, self.runtime_env, self.username, self.password)
        

        self.experiment = super().get_experiment(name=self.experiment_name)
        self.experiment_id = self.experiment.id
        self.experiment_name = self.experiment.name

        self.channel_info = self.experiment.get()['channels']
        self.channel_id_list = list(map(lambda x: x.get()['screen_id'], self.channel_info))
        self.channel_id_list_join = "'" + "','".join(self.channel_id_list) + "'"
        
        # self.list_buckets = super().list_buckets(self.list_buckets)
        self.list_buckets = super().list_buckets(self.experiment)
        
        self.bucket_id2name = {b.id : b.name for b in self.list_buckets}
        self.bucket_name2id = {b.name : b.id for b in self.list_buckets}

        from datetime import datetime, date, timedelta
        from dateutil.relativedelta import relativedelta
        self.TDAY = datetime.today()

        self.DT = self.TDAY.strftime("%Y-%m-%d")
        self.DT_NODASH = self.TDAY.strftime("%Y%m%d")

        self.BF_D1_DT = (self.TDAY - timedelta(days=1)).strftime("%Y-%m-%d")
        self.BF_D30_DT = (self.TDAY - timedelta(days=30)).strftime("%Y-%m-%d")
        self.BF_D90_DT = (self.TDAY - timedelta(days=90)).strftime("%Y-%m-%d")

        self.BF_D10_DT = (self.TDAY - timedelta(days=10)).strftime("%Y-%m-%d")
        self.BF_D10_DT_NODASH = (self.TDAY - timedelta(days=10)).strftime("%Y%m%d")

        
    # (1) 실험Client 정보 호출

    def get_experiment_info(self):
        return {"id" : self.experiment_id, "name" : self.experiment_name}

    def get_bucket_info(self):
        bucket_info = [{"id" : b.id, "name" : b.name, "created" : b.created_at.strftime('%Y%m%d'), "range" : b.bucket_range} for b in self.list_buckets]
        bucket_info = [dict(b, ratio=int(b["range"].split('~')[1]) - int(b["range"].split('~')[0]) + 1) for b in bucket_info]
        bucket_info = [b for b in sorted(bucket_info, key=lambda x: x['created'], reverse=False)]

        return bucket_info

        # (2) Thompson Sampling 실행

    def get_data_from_gcp(self):
        self.qry = f"""
            SELECT
                dt,
                channel_id,
                bucket_name,
                sum(impression_yn) as denominator,
                sum(click_yn) as numerator,
                sum(click_yn) / sum(impression_yn) * 100 as CTR
            FROM
                scm.mls_reco_conv_scrb_daily
            WHERE
                ci_api_distinct_idx = 1
                and impression_yn = 1
                
                and channel_id in ({self.channel_id_list_join})
            GROUP BY
                dt,
                channel_id,
                bucket_name
            ;
        """
        print(self.qry)
        self.df = bq_to_pandas(self.qry)
        self.df['dt'] = pd.to_datetime(self.df['dt'])
        self.df['channel_id'] = self.df['channel_id'].astype(str)
        self.df['bucket_name'] = self.df['bucket_name'].astype(str)
        
        self.key = ['channel_id', 'bucket_name', 'dt']
        self.key_ = ['channel_id', 'bucket_name']
        
        self.df = self.df.reset_index().sort_values(self.key).reset_index(drop=True).copy(deep=True)
        
        self.df['denominator_w7'] = self.df.groupby(self.key_)['denominator'].transform(lambda x: x.rolling(window=7, min_periods=1, center=False).sum())
        self.df['numerator_w7'] = self.df.groupby(self.key_)['numerator'].transform(lambda x: x.rolling(window=7, min_periods=1, center=False).sum())

        self.df['denominator_w14'] = self.df.groupby(self.key_)['denominator'].transform(lambda x: x.rolling(window=14, min_periods=1, center=False).sum())
        self.df['numerator_w14'] = self.df.groupby(self.key_)['numerator'].transform(lambda x: x.rolling(window=14, min_periods=1, center=False).sum())
        
        self.df['CTR_w7'] = self.df['numerator_w7'] / self.df['denominator_w7']
        self.df['CTR_w14'] = self.df['numerator_w14'] / self.df['denominator_w14']
        
        self.df = self.df.loc[self.df['dt'] == self.BF_D1_DT]
        return self.df

    def run_thompson(self):
       
        import pandas as pd
        self.df_result = dict(zip(['A', 'B', 'GA', 'Z'], [20, 15, 55, 10]))
        
        return self.df_result

    def report_thompson(self, df, slack_channel='#ep_report_test'):
        colors = ['rgba(207, 27, 30, .4)',
                  'rgba(15, 38, 214, .4)',
                  'rgba(10, 108, 94, .4)',
                  'rgba(147, 207, 27, .4)',
                  'rgba(10, 5, 0, .4)',
                  'rgba(117, 15, 140, .4)']
        %matplotlib inline

        x = np.linspace(0.005,0.03,1000)

        for m in range(df.shape[0]):
            print('test1----------------------------- :')
            print(df.shape[0])
            
            s = 'self.trace' + str(m)
            print(s)
            a = df.loc[df.bucket_name == df.bucket_name.values[m]].numerator_w7
            b = df.loc[df.bucket_name == df.bucket_name.values[m]].denominator_w7-df.loc[df.bucket_name == df.bucket_name.values[m]].numerator_w7

            y = beta.pdf(x, 1 + a, 1 + b)
            y /= y.max() ## normalize

            exec(s + f""" = go.Scatter(x=x,
                            y=y,
                            name='Bucket {df.bucket_name.values[m]}',
                            marker = dict(color=('rgba(10, 108, 94, 1)')),
                            fill='tozeroy',
                            fillcolor = '""" + colors[m] + f"""')""")

        fig = go.Figure([self.trace0, self.trace1, self.trace2, self.trace3])
        fig.update_layout(
        # title='Beta Distributions for Buckets',
        xaxis={'title': 'Possible CTR values'},
        yaxis={'title': 'Probability Density'},
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ))
        # fig.savefig("./temp/result_2.png")
        fig.show()
        fig.write_image(f"./temp/result_{self.env}_{self.experiment_name}_1.png", height = 300) 

        import matplotlib.pyplot as plt

        data = df
        header_columns=0
        col_width=2.0
        row_height=0.625
        font_size=14
        header_color='#40466e'
        row_colors=['#f1f1f2', 'w']
        edge_color='w'
        bbox=[0, 0, 1, 1]
        ax=None

        if ax is None:
            size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
            fig, ax = plt.subplots(figsize=size)
            ax.axis('off')
        mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns)
        mpl_table.auto_set_font_size(False)
        mpl_table.set_fontsize(font_size)

        for k, cell in mpl_table._cells.items():
            cell.set_edgecolor(edge_color)
            if k[0] == 0 or k[1] < header_columns:
                cell.set_text_props(weight='bold', color='w')
                cell.set_facecolor(header_color)
            else:
                cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
        
        fig = ax.get_figure()


        fig.savefig(f"./temp/result_{self.env}_{self.experiment_name}_2.png")

        
        # Slack package
        from slack_sdk import WebClient
        from slack_sdk.errors import SlackApiError
        from skt.vault_utils import get_secrets
        token = get_secrets("slack")["bot_token"]["airflow"]
        proxy = get_secrets("proxy")["proxy"]
        proxies = {
            "http": proxy,
            "https": proxy,
        }
        headers = {
            "Content-Type": "application/json;charset=utf-8",
            "Authorization": f"Bearer {token}",
        }

        client = WebClient(token=token, proxy=proxy)

        #---------------------------------------------------------------------------#
        ##### TITLE
        response = client.files_upload(file= "./temp/bucket_v5.png",
                                       channels = '#ep_report_test',
                                       initial_comment = f""":calendar: *{self.BF_D1_DT}* ( today : {self.DT} )""",
                                          title = '　　　　　　　　　　　　　　　　　　　　Author : 홍석진, 이준혁')

        #---------------------------------------------------------------------------#
        # RESULT 1
        #---------------------------------------------------------------------------#
        response = client.files_upload(file= f"./temp/result_{self.env}_{self.experiment_name}_2.png",
                                       channels = slack_channel,
                                       initial_comment = f"""
        :bulb: 신규 Bucket 추가
        :bulb: Bucket Ratio 순위 변동
        """,
                                          title = 'Bucket별 Beta 분포')



        ##### RESULT 2
        response = client.files_upload(file= f"./temp/result_{self.env}_{self.experiment_name}_1.png",
                                       channels = slack_channel,
                                       initial_comment = f"""
        """,
                                          title = '일자별 Bucket별 Ratio')



# def get_daily_ctr(experiment_info):
#     """Calculates daily CTR of each buckets in the experiment.
#     Args:
#         experiment_info (dict): Experiment summary. Use output of the `desc_experiment` function.
#     Returns:
#         pd.DataFrame
#             Dataframe daily CTR info. Data columns are as follows:
#             ===================  ==============================================================
#             dt                   date (as `datetime`)
#             channel_id           MLS channel ID (as `str`)
#             bucket_name          MLS bucket name (as `str`)
#             denominator          impression count (as `int`)
#             numerator            click count (as `int`)
#             CTR                  Click-Through Rate
#             bucket_of_interest   1 if bucket is related to the MLFlow experiment, else 0
#             ===================  ==============================================================
#     """

#     channel_id_list = get_channels_from_experiment(experiment_info['연결실험이름'])
#     channel_id_list_join = "'" + "','".join(channel_id_list) + "'"
#     qry = f"""
#     SELECT
#       dt,
#       channel_id,
#       bucket_name,
#       sum(impression_yn) as denominator,
#       sum(click_yn) as numerator,
#       sum(click_yn) / sum(impression_yn) * 100 as CTR
#     FROM
#       scm.mls_reco_conv_scrb_daily
#     WHERE
#       ci_api_distinct_idx = 1
#       and impression_yn = 1
#       and dt >= '{experiment_info['실험시작일']}'
#       and channel_id in ({channel_id_list_join})
#     GROUP BY
#       dt,
#       channel_id,
#       bucket_name
#       ;
#       """
#     df = bq_to_pandas(qry)
#     df['dt'] = pd.to_datetime(df['dt'])
#     df['channel_id'] = df['channel_id'].astype(str)
#     df['bucket_name'] = df['bucket_name'].astype(str)
#     df['bucket_of_interest'] = np.where(df['bucket_name'] == experiment_info['연결버킷명'], 1, 0)
#     return df
        
#         def add_data(self, df, bucket_colname, success_colname, fail_colname):
#         # """
#         # 톰슨샘플링 실행할 때 입력될 수 있지만, 사전에 등록해서 데이터 검증 진행
#         # """
#             self.bucket_colname = bucket_colname
#             self.success_colname = success_colname
#             self.fail_colname = fail_colname

#             assert bucket_colname not in thompson_df, f"{bucket_colname}이 데이터에 존재하지 않습니다."
#             assert success_colname not in thompson_df, f"{success_colname}이 데이터에 존재하지 않습니다."
#             assert fail_colname not in thompson_df, f"{bufail_colnamecket_colname}이 데이터에 존재하지 않습니다."
#             self.thompson_df = df[[bucket_colname, success_colname, fail_colname]]

#             data_bucket = list(thompson_df[bucket_colname])
#             assert len(data_bucket) != len(set(data_bucket)), "버킷별 데이터가 단일하지 않습니다."

#         return self.thompson_df

#         def run_thompson(self, sample_size):
#             for bucket in self.bucket_name2id:
#                 if bucket in self.thompson_df[self.bucket_colname] :
#                     tmp = self.thompson_df[self.thompson_df[self.bucket_colname] == bucket].reset_index(drop=True)
#                     sucess =tmp[self.success_colname][0]
#                     fail =tmp[self.fail_colname][0]

#                     beta_rvs_probs = list(beta.rvs(success, fail, size=sample_size)
    
#                 else :
#                     print('')
                                


        # def report_thompson(self):




        # (3) Thompson Sampling 시뮬레이션

        # def add_data_simul(self, df, bucket_colname, success_colname, fail_colname, series_colname)



        # (4) 버킷 업데이트 및 MLS 결과 확인
        # def update_bucket


In [ ]:
from sktmls import MLSENV, MLSRuntimeENV, MLSClient, ModelRegistry
from sktmls.experiments.experiment import Experiment, ExperimentClient

In [ ]:
from skt.gcp import bq_to_pandas

In [ ]:
### STG
jhjh = mls_thompson(env=MLSENV.STG, 
                    runtime_env=MLSRuntimeENV.YE, 
                    username="scm", 
                    password="rnehrahebf", 
                    experiment_name = 'battleground_jhjh_test')

In [ ]:
### PRD
jhjh = mls_thompson(env=MLSENV.PRD, 
                    runtime_env=MLSRuntimeENV.YE, 
                    username="scm", 
                    password="rnehrahebf", 
                    experiment_name = 'scm_reco_subsc_prod')


In [ ]:
jhjh.get_experiment_info()

In [ ]:
jhjh.get_bucket_info()

In [ ]:
dat = jhjh.get_data_from_gcp()

In [ ]:
dat

In [ ]:
result = jhjh.run_thompson()
result

In [ ]:
jhjh.report_thompson(df = dat)

In [ ]:
result['ratio2'] = round(result['ratio']*(10*dat.shape[0]))

In [ ]:
result

In [ ]:
dat0 = dat['bucket_name'].copy(deep=True).pd
dat0['base_ratio'] = 10

In [ ]:
dat0

In [ ]:
result2 = {'bucket_name' : list(result.bucket_name.values),
           'bucket_ratio' : list(result.ratio*(10*dat.shape[0]))}

In [ ]:
result2

In [ ]:
dict(zip(list(result.bucket_name.values), list(result.ratio.values)))

In [ ]:
data = {'dt' : jh_tdt,
        'no' : jh_i,
        'bucket_name': jh,
        'base': jh_r,
        'update_dt' : jh_udt}
data2 = pd.DataFrame(data)
print(data2)

data3 = {'bucket_name' : dat_jh.bucket_name,
         'bucket_ratio' : np.round(np.array(R)/sum(R)*(100-(10*len(my_bucket))))
        }
data4 = pd.DataFrame(data3)
print(data4)

In [ ]:
for e in jhjh.list_buckets:
    print(e.name)

In [ ]:
experiment_client = ExperimentClient(env=MLSENV.STG, 
                                     runtime_env=MLSRuntimeENV.YE, 
                                     username="scm", 
                                     password="rnehrahebf")

In [ ]:
?experiment_client.list_buckets

In [ ]:
experiment_client.list_experiments()

In [ ]:
# origin workflow

In [ ]:
from sktmls.experiments.experiment import ExperimentClient # Bucket, Experiment, 

In [ ]:
from sktmls import MLSENV, MLSRuntimeENV

from skt.mls import get_mls_config
config = get_mls_config(env='stg', user='scm')
config

# ?get_mls_config

In [ ]:
experiment_client = ExperimentClient(**config)

In [ ]:
experiments = experiment_client.list_experiments()
for e in experiments:
    print(e.name)

In [ ]:
target_experiment = 'battleground_jhjh_test'
target_experiment

In [ ]:
my_experiment = experiment_client.get_experiment(name=target_experiment)

In [ ]:
my_experiment

In [ ]:
my_experiment.updated_at

In [ ]:
# get mls config

In [ ]:
import concurrent.futures

In [ ]:
env = 'dev'
user = 'scm'

In [ ]:

configs = generate_configs(env=env, user=user)
configs

In [ ]:
e = concurrent.futures.ThreadPoolExecutor(max_workers=len(configs) + 1)
fs = [e.submit(check_client_config, conf) for conf in configs]
for f in concurrent.futures.as_completed(fs):
if f.exception() is None:
config = f.result()
break
e.shutdown(wait=False)
return config

In [ ]:
####################################################
# dataframe 및 수치연산을 위한 library load
import pandas as pd
import numpy as np
import os
from datetime import datetime
####################################################
# 그래프를 그리기 위한 library 로드 (matplotlib / seaborn)
from matplotlib import pyplot as plt
import seaborn as sns # matplotlib 기반의 visualization library 
plt.style.use(['ggplot'])

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus']=False
# NanumGothic 폰트가 안나오는 현상을 제거하기 위해 cache 업데이트
plt.rcParams["font.family"] = 'NanumGothic'
sns.set_style("darkgrid", {"font.family":['NanumGothicCoding']}) # 배경색 및 한글론트 설정
sns.set_palette("deep")
%matplotlib inline

####################################################
from datetime import date
import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
pd.set_option('display.max_columns', 100)

from skt.gcp import bq_to_pandas, df_to_bq_table, bq_insert_overwrite 
from skt.gcp import get_bigquery_client, load_bigquery_ipython_magic
from skt.ye import slack_send

load_bigquery_ipython_magic()

In [ ]:
# skt mls api 
import time
from skt.mls import get_mls_config, get_mls_component_client, get_mls_experiment_client
from skt.mls import get_mls_model_registry, get_mls_dimension_client, get_mls_ml_model_client, get_mls_profile_api_client

from sktmls import MLSENV, MLSRuntimeENV, MLSClient, ModelRegistry
from sktmls.filters.filter import FilterClient
from sktmls.datasets import DatasetClient, ProblemType, FeatureStoreConf, LabelDataConf
from sktmls.models import MLModelClient
from sktmls.ml_features import MLFeatureClient
from sktmls.experiments.experiment import Bucket, Experiment, ExperimentClient

In [ ]:
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
TDAY = datetime.today()

DT = TDAY.strftime("%Y-%m-%d")
DT_NODASH = TDAY.strftime("%Y%m%d")

BF_D1_DT = (TDAY - timedelta(days=1)).strftime("%Y-%m-%d")
BF_D30_DT = (TDAY - timedelta(days=30)).strftime("%Y-%m-%d")
BF_D90_DT = (TDAY - timedelta(days=90)).strftime("%Y-%m-%d")

BF_D10_DT = (TDAY - timedelta(days=10)).strftime("%Y-%m-%d")
BF_D10_DT_NODASH = (TDAY - timedelta(days=10)).strftime("%Y%m%d")

print("DT : ", DT)
print("DT_NODASH : ", DT_NODASH)

print("BF_D1_DT : ", BF_D1_DT)
print("BF_D30_DT : ", BF_D30_DT)

print("BF_D10_DT : ", BF_D10_DT)
print("BF_D10_DT_NODASH : ", BF_D10_DT_NODASH)

<br><br>

### 001_기본 설정 

In [ ]:
from sktmls import MLSENV, MLSRuntimeENV

from skt.mls import get_mls_config
config = get_mls_config(env='stg', user='scm')
config

In [ ]:
experiment_client = ExperimentClient(**config)

<br><br>

### 002_내 실험 찾기, 그냥 입력해도 되고

In [ ]:
experiments = experiment_client.list_experiments()

In [ ]:
for e in experiments:
    print(e.name)

In [ ]:
target_experiment = experiments[-1].name
target_experiment

In [ ]:
target_experiment = 'battleground_jhjh_test'
target_experiment

<br><br>

### 002_실험 불러오기

In [ ]:
my_experiment = experiment_client.get_experiment(name=target_experiment)

In [ ]:
for e in dir(my_experiment):
    print(e)
    print

<br><br>

### 003_버킷 리스트 불러오기

In [ ]:
my_bucket = experiment_client.list_buckets(
    experiment=my_experiment
)

In [ ]:
print('--------------------------------------------------------')
for e in my_bucket:
    print('- 버킷명 : ' + e.name)
    print('- ID : ' + str(e.id))   
    print('- 버킷 비율 : ' + e.bucket_range + ' <--- 중요, 전체 100% 버킷중 비율 의미')    
    print('- 세부내용 : ' + str(e.description))    
    print('- 실험명 : ' + e.experiment.name) 
    print('--------------------------------------------------------')


In [ ]:
my_bucket

<br><br>

### 999_Thompson Sampling 추가
---
- Thompson sampling을 적용하더라도, 각 bucket별 상한, 하한은 존재해야 할것이며,
  세부 항목은 우리가 정하겠지?

- 그 안에서 자동으로 Bucket Ratio를 조절하고, 그로 인한 실적 향상을 측정해보는 것에 의의가 있음
---
> **[ 예시 ]** <br><br>&nbsp;&nbsp;&nbsp;&nbsp;**<span style='color:red'>A (BTS v1)</span>**&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;**<span style='color:blue'>B (BTS v2)</span>**&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;**<span style='color:black'>Z (Random)</span>**<br><br>위와 같이 3개의 bucket이 있을때,<br>
  각각 70%, 20%, 10% 고정이 아닌 50%, 10%, 10%는 고정해둔 상태에서 <br>
  A와 B에 대하여 나머지 30%의 비율을 적절하게 배분하여 추천하는 방식을 채택한다면?


<br>

#### Beta 분포 업데이트를 위하여 일자별 버킷별 불러오기 ( w/ 실험 플랫폼 )

In [ ]:
import os
os.system('mkdir -p battleground')
os.system('mkdir -p temp')

In [ ]:
from skt.vault_utils import get_secrets
from skt.github_utils import GithubUtil
import io

path = f"https://github.com/sktaiflow/dag-advanced_analytics/blob/develop/experiment_platform/battleground/experiment"
files = [
        f'monitoring_report.py'
        ]

secrets = get_secrets('github/sktaiflow')
token = secrets['token']

proxies = {
    'http': secrets['proxy'],
    'https': secrets['proxy']
    }

g = GithubUtil(token, proxies=proxies)
for file in files:
    code = g.download_from_git(f"{path}/{file}").decode('utf-8')
    with open(f"battleground/{file}", "w+") as f:
        f.write(code)



In [ ]:
from skt.gcp import bq_to_pandas
from skt.ye import slack_send
from datetime import datetime, date
from battleground.monitoring_report import desc_experiment, get_channels_from_experiment, get_daily_ctr, plot_performance, send_slack, update_experiment_end_dt, create_report

#### 우리가 테스트 하고자하는 실험의 어떤 버켓이던 상관없을듯. 
---
- 나는 우선 BTS V2를 예시로 선택
---

In [ ]:
experiment_nm = '[exp001]galileo_single_reco_model'

In [ ]:
experiment_info = desc_experiment(experiment_nm)

In [ ]:
experiment_info

In [ ]:
df = get_daily_ctr(experiment_info)

In [ ]:
# from datetime import datetime
# df.dt.strftime("%V")

In [ ]:
df.loc[df.dt <= '2022-04-01']

In [ ]:
key = ['channel_id', 'bucket_name', 'dt']
key_ = ['channel_id', 'bucket_name']

In [ ]:
stat = df.reset_index().sort_values(key).reset_index(drop=True).copy(deep=True)

In [ ]:
stat['denominator_w7'] = stat.groupby(key_)['denominator'].transform(lambda x: x.rolling(window=7, min_periods=1, center=False).sum())
stat['numerator_w7'] = stat.groupby(key_)['numerator'].transform(lambda x: x.rolling(window=7, min_periods=1, center=False).sum())

stat['denominator_w14'] = stat.groupby(key_)['denominator'].transform(lambda x: x.rolling(window=14, min_periods=1, center=False).sum())
stat['numerator_w14'] = stat.groupby(key_)['numerator'].transform(lambda x: x.rolling(window=14, min_periods=1, center=False).sum())
#
# stat['click_cnt_w7'] = stat.groupby(key_)['click_cnt'].transform(lambda x: x.rolling(window=7, min_periods=1, center=False).sum())
# stat['scrb_cnt_w7'] = stat.groupby(key_)['scrb_cnt'].transform(lambda x: x.rolling(window=7, min_periods=1, center=False).sum())

In [ ]:
stat['CTR_w7'] = stat['numerator_w7'] / stat['denominator_w7']
stat['CTR_w14'] = stat['numerator_w14'] / stat['denominator_w14']

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import beta, bernoulli
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots
import random
import math

In [ ]:
dat_jh = stat.loc[stat.dt == BF_D10_DT].copy(deep=True) #BF_D10_DT #BF_D1_DT

In [ ]:
dat_jh

In [ ]:
ads = list(dat_jh.bucket_name)
ACTUAL_CTR = dict(zip(dat_jh.bucket_name, dat_jh.CTR))
ads_pass =list(dat_jh.denominator)
ads_fail =list(dat_jh.denominator - dat_jh.numerator)

R = [0 for i in range(len(ads))]


In [ ]:
###new

for trial in range(10000):
    # print("\nTrial " + str(trial))

    rnd = np.random.RandomState()  # for machine payouts and Beta

    probs = []

    for m in range(len(ads)):
        # print(m)
        # probs[m] = rnd.beta(1 + ads_pass[m], 1 +  ads_fail[m])
        dat_jh['probs'] = rnd.beta(dat_jh['numerator_w7'], dat_jh['denominator_w7']-dat_jh['numerator_w7'])

    machine = np.argmax(dat_jh.probs)
    
    R[machine] += 1
    
    if trial%1000 == 0:
        print(trial)
        print(dat_jh.probs)
print("\nFinal Success vector: ", end="")
R


In [ ]:
TARGET_DT = BF_D10_DT

In [ ]:
jh = []
jh_i = []
jh_r = []
jh_tdt = []
jh_udt = []
base = 10
target_dt = TARGET_DT
update_dt = DT
for i in range(len(my_bucket)):
    print(i)
    bucket_name_tmp = my_bucket[i].name
    print(bucket_name_tmp)
    
    jh_i += [i]
    jh += [bucket_name_tmp]
    jh_r += [base]
    jh_tdt += [target_dt]
    jh_udt += [update_dt]
    
print(jh)
print(jh_i)
print(jh_r)
print(jh_tdt)
print(jh_udt)

data = {'dt' : jh_tdt,
        'no' : jh_i,
        'bucket_name': jh,
        'base': jh_r,
        'update_dt' : jh_udt}
data2 = pd.DataFrame(data)
print(data2)

data3 = {'bucket_name' : dat_jh.bucket_name,
         'bucket_ratio' : np.round(np.array(R)/sum(R)*(100-(10*len(my_bucket))))
        }
data4 = pd.DataFrame(data3)
print(data4)

In [ ]:
sample_size12 = pd.merge(left = data2 , right = data4, how = "left", on = "bucket_name")

In [ ]:
sample_size12['bucket_ratio2'] = sample_size12['base'] + sample_size12['bucket_ratio'].fillna(0)
sample_size12['bucket_ratio2'] =  sample_size12['bucket_ratio2'].astype(int) 

In [ ]:
sample_size12

In [ ]:
from skt.gcp import pandas_to_bq, pandas_to_bq_table
?pandas_to_bq_table

In [ ]:
pandas_to_bq_table(sample_size12, 'x1112055', 'battleground_report_ts_summary')

In [ ]:
bq = get_bigquery_client()
max_id = bq_to_pandas(f"SELECT * FROM x1112055.battleground_report_ts_summary")
# qry = f"""
# INSERT INTO x1112834.battleground_monitoring_summary(id, experiment_nm, slack_channel, experiment_end_dt)
# VALUES ({1 if pd.isnull(max_id) else max_id+1}, '{experiment_nm}', '{self.slack_channel}', '99991231')
# """
# result = bq.query(query=qry)
# print(result.result())

In [ ]:
max_id

In [ ]:
my_bucket_ratio = dict(zip(my_bucket, sample_size12.bucket_ratio2))

In [ ]:
bucket_list = []
for a in my_bucket:
    bucket_list += [a.name]
bucket_list

In [ ]:
if bucket_list == list(sample_size12.bucket_name):
    print('True')
else:
    prtin('Stop')

In [ ]:
my_bucket_ratio

In [ ]:
# dat_jh.loc[dat_jh.bucket_name == ads[m]].numerator_w7
# dat_jh.loc[dat_jh.bucket_name == ads[m]].denominator_w7-dat_jh.loc[dat_jh.bucket_name == ads[m]].numerator_w7

In [ ]:
colors = ['rgba(207, 27, 30, .4)',
'rgba(15, 38, 214, .4)',
'rgba(10, 108, 94, .4)',
'rgba(147, 207, 27, .4)',
'rgba(10, 5, 0, .4)',
'rgba(117, 15, 140, .4)']

In [ ]:
# pip install -U kaleido

In [ ]:
%matplotlib inline

In [ ]:
## plot the Beta distributions
x = np.linspace(0.005,0.03,1000)

 
# x = np.linspace(0.01,0.04,1000)

for m in range(len(ads)):
    
    s = 'trace' + str(m)
    print(ads[m])
    a = dat_jh.loc[dat_jh.bucket_name == ads[m]].numerator_w7
    b = dat_jh.loc[dat_jh.bucket_name == ads[m]].denominator_w7-dat_jh.loc[dat_jh.bucket_name == ads[m]].numerator_w7
    
    print(a)
    print(b)
    
    y = beta.pdf(x, 1 + a, 1 + b)

    y /= y.max() ## normalize

    exec(s + f""" = go.Scatter(x=x,
                        y=y,
                        name='Bucket {ads[m]}',
                        marker = dict(color=('rgba(10, 108, 94, 1)')),
                        fill='tozeroy',
                        fillcolor = '""" + colors[m] + f"""')""")

fig = go.Figure([trace0, trace1, trace2, trace3])
fig.update_layout(
    # title='Beta Distributions for Buckets',
    xaxis={'title': 'Possible CTR values'},
    yaxis={'title': 'Probability Density'},
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))


# fig.savefig("./temp/result_2.png")
fig.write_image("./temp/result_2.png", height = 300) 
fig.show()


In [ ]:
my_experiment

In [ ]:
experiment_client.update_bucket_ratio(my_experiment, my_bucket_ratio)

In [ ]:
my_bucket = experiment_client.list_buckets(
    experiment=my_experiment
)

In [ ]:
print('--------------------------------------------------------')
for e in my_bucket:
    print('- 버킷명 : ' + e.name)
    print('- ID : ' + str(e.id))   
    print('- 버킷 비율 : ' + e.bucket_range + ' <--- 중요, 전체 100% 버킷중 비율 의미')    
    print('- 세부내용 : ' + str(e.description))    
    print('- 실험명 : ' + e.experiment.name) 
    print('--------------------------------------------------------')


# 저장 예시

In [ ]:
dat_jh

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

fig,ax = render_mpl_table(max_id, header_columns=0, col_width=2.0)
fig.savefig("./temp/result_1.png")

# Slack 전송

In [ ]:

# Slack package
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

In [ ]:
token = get_secrets("slack")["bot_token"]["airflow"]
proxy = get_secrets("proxy")["proxy"]
proxies = {
    "http": proxy,
    "https": proxy,
}
headers = {
    "Content-Type": "application/json;charset=utf-8",
    "Authorization": f"Bearer {token}",
}

In [ ]:
client = WebClient(token=token, proxy=proxy)

In [ ]:
import os
os.getcwd()

In [ ]:
datdat = stat.loc[(stat.dt >= BF_D10_DT) & (stat.channel_id =='scm_reco_subsc_prod')].copy(deep=True) #BF_D10_DT #BF_D1_DT

In [ ]:
pd.DataFrame(ratio_series_adj1).plot(color=color_info)

In [ ]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)

d = {'col1': [1, 2], 'col2': [3, 4], 'col3': ['a', 'b']}
df2 = pd.DataFrame(data=d)

d = {'col2': [30, 4], 'col3': ['a', 'b'], 'col1': [111111, 2]}
df3 = pd.DataFrame(data=d)

df12 = pd.concat([df,df2, df3])

In [ ]:
pandas_to_bq_table(sample_size12, 'tos_auto_mkt', f'mthl_mkt_{chnl}_lchn_target_control_tbl')

bq_insert_overwrite(destination=f"sktaic-datahub.x1112055.battleground_report_ts_summary", sql=qry_2_0)

# bq = get_bigquery_client()
# qry = f"""
# create table x1112055.battleground_report_ts_summary

# """
# result = bq.query(query=qry)
# print(result.result())

In [ ]:
insert into  di_mi.mov_w_idx (home_loc_cd, home_loc_nm, age_cd, sex_cd, id_cnt, h0_mov_dist, h1_mov_dist, h0_max_h_dist, h1_max_h_dist, loc_type, yw) 
SELECT home_loc_cd, home_loc_nm, age_cd, sex_cd, id_cnt, h0_mov_dist, h1_mov_dist, h0_max_h_dist, h1_max_h_dist, loc_type, CAST(yw as INT64) as yw
FROM x1112055.mov_w_idx_nonpartition

In [ ]:
# table update hsh 
#----
# bq = get_bigquery_client()
#     qry = f"""
#     UPDATE x1112834.battleground_monitoring_summary SET experiment_end_dt = '{date.today().strftime('%Y%m%d')}' WHERE id = {id}
#     """
#     result = bq.query(query=qry)
#     print(result.result())

In [ ]:
df12

In [ ]:
datdat.set_index('dt', inplace=True)
datdat.groupby('bucket_name')['CTR'].plot(legend=True)

# ㅇㅇㅇ

In [ ]:
#---------------------------------------------------------------------------#
##### TITLE
response = client.files_upload(file= "./temp/bucket_v5.png",
                               channels = '#ep_report_test',
                               initial_comment = f""":calendar: *{BF_D10_DT}* ( today : {DT} )""",
                                  title = '　　　　　　　　　　　　　　　　　　　　Author : 홍석진, 이준혁')

#---------------------------------------------------------------------------#
# RESULT 1
#---------------------------------------------------------------------------#
response = client.files_upload(file= "./temp/result_2.png",
                               channels = '#ep_report_test',
                               initial_comment = f"""
:bulb: 신규 Bucket 추가
:bulb: Bucket Ratio 순위 변동
""",
                                  title = 'Bucket별 Beta 분포')



##### RESULT 2
response = client.files_upload(file= "./temp/result_1.png",
                               channels = '#ep_report_test',
                               initial_comment = f"""
""",
                                  title = '일자별 Bucket별 Ratio')


In [ ]:
filepath_list = [
    "./temp/bucket_v4.png",
    "./temp/result_1.png",
    "./temp/result_2.png"
]

link_list = []
i = 0
for filepath in filepath_list:
    response = client.files_upload(file=filepath,channels = '#ep_report_test',
                                  initial_comment = """
                                  
*:muscle: Daily Bucket Report*
- ABC
> ABC
_ABC_
__ABC__
                                  
""",
                                  title = 'Thompson Sampling')
    # link = response.data['file']['permalink']
    # link_list.append(link)

# link_message = ""
# for link in link_list:
#     link_message += f"<{link}| >\n----\n<{link}|aa>\n"

# message = """Daily Bucket Report \n\n
# <span style = "color:red"> 가나다라</span> ## 가다나라
# """
# message += link_message
# client.chat_postMessage(channel="#ep_report_test", text=message)
# # client.chat.postMessage(channel="#ep_report_test", text=message)